In [20]:
# ensure installed packages can be used in the current notebook within the context of the current Python kernel
import sys
!{sys.executable} -m pip install pandas
!{sys.executable} -m pip install lxml
!{sys.executable} -m pip install display
!{sys.executable} -m pip install display_pretty
!{sys.executable} -m pip install Javascript
!{sys.executable} -m pip install setuptools

ERROR: Could not find a version that satisfies the requirement display_pretty (from versions: none)
ERROR: No matching distribution found for display_pretty


In [21]:
import requests
import pandas as pd
import re
import warnings
from lxml import html
from IPython.display import display, display_pretty, Javascript, HTML

warnings.filterwarnings("ignore", message="Unverified HTTPS request")

In [3]:
# get state geography data from the U.S. Census Bureau

# read state file csv and create dataframe
census_states = pd.read_csv(
    'https://www2.census.gov/geo/docs/reference/state.txt',
    delimiter='|',
    header=0,
    usecols=[0,1,2],
    names=['state_fips', 'state_abbr', 'c_state_name'],
    dtype=str
)

# define a list of FIPS codes that includes Alaska and the internally-autonomous entities the U.S. has some control over
suzerainty_fips = ['02','60','66','69','72','74','78']

# filter out Alaska and these Suzerainty entities
census_states = census_states[~census_states.state_fips.isin(suzerainty_fips)]

print(census_states.shape, '\n', census_states.dtypes)
census_states.head(100)

(50, 3) 
 state_fips      object
state_abbr      object
c_state_name    object
dtype: object


,state_fips,state_abbr,c_state_name
0,01,AL,Alabama
2,04,AZ,Arizona
3,05,AR,Arkansas
4,06,CA,California
5,08,CO,Colorado
6,09,CT,Connecticut
7,10,DE,Delaware
8,11,DC,District of Columbia
9,12,FL,Florida
10,13,GA,Georgia


In [4]:
# get county geography data from the U.S. Census Bureau Gazetteer files

# enter base year for the county gazetteer files
census_year = 2024

# base_url where all the county gazetteer files live
gazetteer_url = f"https://www2.census.gov/geo/docs/maps-data/data/gazetteer/{census_year}_Gazetteer/"

# make an http request for the page
page = requests.request(
    method='GET', 
    url=gazetteer_url,
    headers={ "Accept": "application/json" }
)

# parse the page and return a DOM tree
tree = html.fromstring(page.content)

# use XPath to return a list of link texts ('a' elements within the 'table' element) from the DOM
gazatteer_files = tree.xpath('//td/a/text()')

# filter the list to return only county file names
county_files = [c for c in gazatteer_files if re.match(r'.*counties.*\.txt', c)]

# Convert the list to a series
county_files_series = pd.Series(county_files, name="county_file_name")

# Convert series to dataframe
census_gaz_counties = county_files_series.to_frame()

# filter out Alaska and these Suzerainty entities
census_gaz_counties = census_gaz_counties[~census_gaz_counties.county_file_name.str.extract(r'_(\d{2})\.txt')[0].isin(suzerainty_fips)]

print(census_gaz_counties.shape, '\n', census_gaz_counties.dtypes)
census_gaz_counties.head(100)


(50, 1) 
 county_file_name    object
dtype: object


,county_file_name
0,2024_gaz_counties_01.txt
2,2024_gaz_counties_04.txt
3,2024_gaz_counties_05.txt
4,2024_gaz_counties_06.txt
5,2024_gaz_counties_08.txt
6,2024_gaz_counties_09.txt
7,2024_gaz_counties_10.txt
8,2024_gaz_counties_11.txt
9,2024_gaz_counties_12.txt
10,2024_gaz_counties_13.txt


In [5]:
# parse county geography data from the U.S. Census Bureau Gazetteer files

# read each county file csv
census_county_files = [
    pd.read_csv(
        gazetteer_url + county_file_name,
        delimiter='\t',
        lineterminator='\n',
        header=0,
        usecols=[1,3],
        names=['geoid', 'c_county_name'],
        dtype=str
    ) for county_file_name in census_gaz_counties['county_file_name']]

# combine into a dataframe
census_counties = pd.concat(census_county_files, ignore_index=True)

# create state FIPS codes from the 5-digit 'geoid'
census_counties['state_fips'] = census_counties['geoid'].str[:2]

print(census_counties.shape, '\n', census_counties.dtypes)
census_counties.head(100)

(3114, 3) 
 geoid            object
c_county_name    object
state_fips       object
dtype: object


,geoid,c_county_name,state_fips
0,01001,Autauga County,01
1,01003,Baldwin County,01
2,01005,Barbour County,01
3,01007,Bibb County,01
4,01009,Blount County,01
...,...,...,...
95,05027,Columbia County,05
96,05029,Conway County,05
97,05031,Craighead County,05
98,05033,Crawford County,05


In [6]:
# merge Census counties and states
# Perform a left join on 'state_fips'
county_state_df = census_counties.merge(
    census_states[['state_fips', 'c_state_name']], 
    on='state_fips', 
    how='left'
)

# Drop the 'state_fips' column
census_geographies = county_state_df.drop(columns=['state_fips'], axis=1)

print(census_geographies.shape, '\n', census_geographies.dtypes)
census_geographies.head(100)

(3114, 3) 
 geoid            object
c_county_name    object
c_state_name     object
dtype: object


,geoid,c_county_name,c_state_name
0,01001,Autauga County,Alabama
1,01003,Baldwin County,Alabama
2,01005,Barbour County,Alabama
3,01007,Bibb County,Alabama
4,01009,Blount County,Alabama
...,...,...,...
95,05027,Columbia County,Arkansas
96,05029,Conway County,Arkansas
97,05031,Craighead County,Arkansas
98,05033,Crawford County,Arkansas


In [7]:
# Helper function to format state names for URLs
def format_state(state):
    return state.replace(".", "").replace(",", "").replace(" ", "-").lower()

# Function to get state results from the New York Times
def get_nyt_state_results(state):
    # Request JSON data for the specified formatted state
    formatted_state = format_state(state)
    url = f"https://static01.nyt.com/elections-assets/pages/data/2024-11-05/results-{formatted_state}-president.json"
    response = requests.get(url)
    data = response.json()
    
    # Initialize dataframe and extract races
    races = data.get("races", [])
    rows = []
    
    for race in races:
        for unit in race.get("reporting_units", []):
            if unit.get("fips_county") is not None:
                for candidate in unit.get("candidates", []):
                    row = {
                        "fips_state": unit.get("fips_state"),
                        "fips_county": unit.get("fips_county"),
                        "total_votes": unit.get("total_votes"),
                        "nyt_id": candidate.get("nyt_id"),
                        "total": candidate.get("votes", {}).get("total", 0)
                    }
                    rows.append(row)
    
    # Convert rows to a dataframe
    df = pd.DataFrame(rows)
    
    # Aggregate some states' data to county level
    if formatted_state in ["connecticut","vermont","maine","massachusetts","new-hampshire","rhode-island","washington-dc"]:
        df = df.groupby(["nyt_id", "fips_state", "fips_county"], as_index=False).agg({
            "total": "sum",
            "total_votes": "sum"
        })
    
    # Pivot the data so each candidate's votes are in their own column
    df_pivoted = df.pivot_table(index=["fips_state", "fips_county", "total_votes"], columns="nyt_id", values="total", fill_value=0)
    df_pivoted.reset_index(inplace=True)
    
    # Combine fips_state and fips_county columns to create county_fips
    df_pivoted["county_fips"] = df_pivoted["fips_state"].astype(str) + df_pivoted["fips_county"].astype(str)
    
    return df_pivoted

In [8]:
# Function to get state results from Fox News
def get_fox_state_results(state_abbr):
    # Request JSON data for the specified state
    url = f"https://feeds-elections.foxnews.com/archive/politics/elections/2024/3/2024_Generals/President/{state_abbr}/county_level_results/file.json"
    response = requests.get(url)
    data = response.json()

    # Process lookup table
    lookup_counties = data['lookup']['counties']
    # Convert JSON to dataframe
    lookup_df = pd.DataFrame.from_dict(lookup_counties, orient='index').reset_index(drop=False)
    # Drop the "number" and "precinctsReporting" columns
    lookup_df = lookup_df.drop(columns=["name", "number", "precinctsReporting"])
    lookup_df = lookup_df.rename(columns={'index': 'countyId', 'fipsCode': 'county_fips'})
    # Convert countyId to integer and fipsCode to string
    lookup_df['countyId'] = lookup_df['countyId'].astype(int)
    lookup_df['county_fips'] = lookup_df['county_fips'].astype(str)
    # pad with leading zeros
    lookup_df['county_fips'] = lookup_df['county_fips'].str.zfill(5)

    # Process results table
    results = data['results']
    # Flatten the JSON
    rows = []
    for result in results:
        party_name = result['candidate']['partyName']
        for vote_data in result['countyVotes']:
            county_id = vote_data['countyId']
            vote_count = vote_data['votes']['count']
            row = {
                 'partyName': party_name, 
                 'countyId': county_id, 
                 'count': vote_count
            }
            rows.append(row)

    # Create the dataframe
    cadidate_df = pd.DataFrame(rows)
    
    # Group by county and calculate total votes
    grouped_by_county = cadidate_df.groupby('countyId', as_index=False)['count'].sum().rename(columns={'count': 'total_votes'})
    
    # Filter results for Democrat and Republican candidates only
    filtered_df = cadidate_df[cadidate_df['partyName'].isin(['Democrat', 'Republican'])]
    pivoted_df = filtered_df.pivot(index='countyId', columns='partyName', values='count').reset_index().fillna(0)
    pivoted_df = pivoted_df.rename(columns={'Democrat': 'votes_dem', 'Republican': 'votes_gop'})

    # Merge total votes and pivoted results
    results_df = pivoted_df.merge(grouped_by_county, on='countyId', how='left')

    # Merge with lookup to get FIPS codes
    results_lookup_df = results_df.merge(lookup_df, on='countyId', how='left')

    # Reorder columns
    results_lookup_df = results_lookup_df[['county_fips', 'total_votes', 'votes_dem', 'votes_gop']]
    
    return results_lookup_df

In [9]:
# get election results data from the New York Times

# Define candidates and fields
two_party_candidates = ["harris-k", "trump-d"]
fields = ["county_fips", "total_votes"] + two_party_candidates

# Define state names
state_names = ["Washington, D.C." if state == "District of Columbia" else state for state in census_states['c_state_name']]

# Retrieve results for each state
contiguous_nyt_state_results = [get_nyt_state_results(state) for state in state_names]

# Combine all state dataframes
contiguous_nyt_election_results = pd.concat(contiguous_nyt_state_results, ignore_index=True)

# Reorder columns and rename
contiguous_nyt_election_results = contiguous_nyt_election_results[["county_fips", "total_votes"] + two_party_candidates]
contiguous_nyt_election_results = contiguous_nyt_election_results.rename(columns={"harris-k": "votes_dem", "trump-d": "votes_gop"})

# Calculate GOP percentage
contiguous_nyt_election_results["per_gop"] = contiguous_nyt_election_results["votes_gop"] / contiguous_nyt_election_results["total_votes"]

# Calculate DEM percentage
contiguous_nyt_election_results["per_dem"] = contiguous_nyt_election_results["votes_dem"] / contiguous_nyt_election_results["total_votes"]

# Calculate vote difference
contiguous_nyt_election_results["diff"] = contiguous_nyt_election_results["votes_gop"] - contiguous_nyt_election_results["votes_dem"]

# Calculate percentage point difference
contiguous_nyt_election_results["per_point_diff"] = contiguous_nyt_election_results["per_gop"] - contiguous_nyt_election_results["per_dem"]

# Ensure column types are correctly set
contiguous_nyt_election_results["county_fips"] = contiguous_nyt_election_results["county_fips"].astype(str)
contiguous_nyt_election_results["total_votes"] = contiguous_nyt_election_results["total_votes"].astype("Int64")
contiguous_nyt_election_results["votes_dem"] = contiguous_nyt_election_results["votes_dem"].astype("Int64")
contiguous_nyt_election_results["votes_gop"] = contiguous_nyt_election_results["votes_gop"].astype("Int64")

# Sort by county_fips
contiguous_nyt_election_results = contiguous_nyt_election_results.sort_values(by="county_fips").reset_index(drop=True)

print(contiguous_nyt_election_results.shape, '\n', contiguous_nyt_election_results.dtypes)
contiguous_nyt_election_results.head(100)

(3112, 8) 
 nyt_id
county_fips        object
total_votes         Int64
votes_dem           Int64
votes_gop           Int64
per_gop           float64
per_dem           float64
diff              float64
per_point_diff    float64
dtype: object


nyt_id,county_fips,total_votes,votes_dem,votes_gop,per_gop,per_dem,diff,per_point_diff
0,01001,28139,7429,20447,0.726643,0.264011,13018.0,0.462632
1,01003,120973,24763,95144,0.786490,0.204699,70381.0,0.581791
2,01005,9766,4120,5578,0.571165,0.421872,1458.0,0.149293
3,01007,9230,1617,7563,0.819393,0.175190,5946.0,0.644204
4,01009,28024,2569,25271,0.901763,0.091671,22702.0,0.810091
...,...,...,...,...,...,...,...,...
95,05027,7929,2460,5362,0.676252,0.310253,2902.0,0.365998
96,05029,8516,2444,5873,0.689643,0.286989,3429.0,0.402654
97,05031,37183,11190,25117,0.675497,0.300944,13927.0,0.374553
98,05033,23811,4749,18596,0.780984,0.199446,13847.0,0.581538


In [10]:
# get election results data from Fox News

# Define state abbreviations
state_abbrs = [state for state in census_states['state_abbr']]

# Retrieve results for each state
contiguous_fox_state_results = [get_fox_state_results(state_abbr) for state_abbr in state_abbrs]

# Combine all state dataframes
contiguous_fox_election_results = pd.concat(contiguous_fox_state_results, ignore_index=True)

# Calculate GOP percentage
contiguous_fox_election_results["per_gop"] = contiguous_fox_election_results["votes_gop"] / contiguous_fox_election_results["total_votes"]

# Calculate DEM percentage
contiguous_fox_election_results["per_dem"] = contiguous_fox_election_results["votes_dem"] / contiguous_fox_election_results["total_votes"]

# Calculate vote difference
contiguous_fox_election_results["diff"] = contiguous_fox_election_results["votes_gop"] - contiguous_fox_election_results["votes_dem"]

# Calculate percentage point difference
contiguous_fox_election_results["per_point_diff"] = contiguous_fox_election_results["per_gop"] - contiguous_fox_election_results["per_dem"]

# Ensure column types are correctly set
contiguous_fox_election_results["county_fips"] = contiguous_fox_election_results["county_fips"].astype(str)
contiguous_fox_election_results["total_votes"] = contiguous_fox_election_results["total_votes"].astype("Int64")
contiguous_fox_election_results["votes_dem"] = contiguous_fox_election_results["votes_dem"].astype("Int64")
contiguous_fox_election_results["votes_gop"] = contiguous_fox_election_results["votes_gop"].astype("Int64")

# Sort by county_fips
contiguous_fox_election_results = contiguous_fox_election_results.sort_values(by="county_fips").reset_index(drop=True)

print(contiguous_fox_election_results.shape, '\n', contiguous_fox_election_results.dtypes)
contiguous_fox_election_results.head(100)

(3113, 8) 
 county_fips        object
total_votes         Int64
votes_dem           Int64
votes_gop           Int64
per_gop           float64
per_dem           float64
diff                int64
per_point_diff    float64
dtype: object


,county_fips,total_votes,votes_dem,votes_gop,per_gop,per_dem,diff,per_point_diff
0,01001,28139,7429,20447,0.726643,0.264011,13018,0.462632
1,01003,120973,24763,95144,0.786490,0.204699,70381,0.581791
2,01005,9766,4120,5578,0.571165,0.421872,1458,0.149293
3,01007,9230,1617,7563,0.819393,0.175190,5946,0.644204
4,01009,28024,2569,25271,0.901763,0.091671,22702,0.810091
...,...,...,...,...,...,...,...,...
95,05027,7929,2460,5362,0.676252,0.310253,2902,0.365998
96,05029,8516,2444,5873,0.689643,0.286989,3429,0.402654
97,05031,37183,11190,25117,0.675497,0.300944,13927,0.374553
98,05033,23811,4749,18596,0.780984,0.199446,13847,0.581538


In [11]:
# merge election results on to the Census geography data

# election_results should contain: 'county_fips', 'votes_gop', 'votes_dem', 'total_votes'
# census_geographies should contain: 'geoid', 'c_county_name', 'c_state_name'

# Perform a left join on <source>_election_results and census_geographies
results_geographies = pd.merge(
    contiguous_fox_election_results,
    census_geographies,
    left_on="county_fips",
    right_on="geoid",
    how="left"
)

# Rename columns from the joined data
results_geographies.rename(columns={"c_state_name": "state_name", "c_county_name": "county_name"}, inplace=True)

# Reorder columns
election_results = results_geographies[[
    "state_name", "county_fips", "county_name", "votes_gop", "votes_dem", 
    "total_votes", "diff", "per_gop", "per_dem", "per_point_diff"
]]

print(election_results.shape, '\n', election_results.dtypes)
election_results.head(100)

(3113, 10) 
 state_name         object
county_fips        object
county_name        object
votes_gop           Int64
votes_dem           Int64
total_votes         Int64
diff                int64
per_gop           float64
per_dem           float64
per_point_diff    float64
dtype: object


,state_name,county_fips,county_name,votes_gop,votes_dem,total_votes,diff,per_gop,per_dem,per_point_diff
0,Alabama,01001,Autauga County,20447,7429,28139,13018,0.726643,0.264011,0.462632
1,Alabama,01003,Baldwin County,95144,24763,120973,70381,0.786490,0.204699,0.581791
2,Alabama,01005,Barbour County,5578,4120,9766,1458,0.571165,0.421872,0.149293
3,Alabama,01007,Bibb County,7563,1617,9230,5946,0.819393,0.175190,0.644204
4,Alabama,01009,Blount County,25271,2569,28024,22702,0.901763,0.091671,0.810091
...,...,...,...,...,...,...,...,...,...,...
95,Arkansas,05027,Columbia County,5362,2460,7929,2902,0.676252,0.310253,0.365998
96,Arkansas,05029,Conway County,5873,2444,8516,3429,0.689643,0.286989,0.402654
97,Arkansas,05031,Craighead County,25117,11190,37183,13927,0.675497,0.300944,0.374553
98,Arkansas,05033,Crawford County,18596,4749,23811,13847,0.780984,0.199446,0.581538


In [12]:
# get Alaska state geography data from the U.S. Census Bureau

# read state file csv and create dataframe
ak_sldlsts = pd.read_csv(
    'https://www2.census.gov/geo/docs/reference/codes2020/sldl/st02_ak_sldl2020.txt',
    delimiter='|',
    header=0,
    dtype=str
)
ak_sldlsts["geoid"] = ak_sldlsts["STATEFP"].astype(str) + ak_sldlsts["SLDLST"].astype(str)

# Rename columns
ak_sldlsts.rename(columns={"SLDLNAME": "sldlst_name", "STATE": "state_abbr", "STATEFP": "state_fips"}, inplace=True)

# Remove unnecessary columns
ak_sldlsts.drop(columns=["state_abbr", "SLDLST"], inplace=True)

# add Alaska name to state_name column
ak_sldlsts["state_name"] ="Alaska"

# Reorder columns
ak_sldlsts = ak_sldlsts[["state_name", "geoid", "sldlst_name", "state_fips"]]

print(ak_sldlsts.shape, '\n', ak_sldlsts.dtypes)
ak_sldlsts.head(100)

(40, 4) 
 state_name     object
geoid          object
sldlst_name    object
state_fips     object
dtype: object


,state_name,geoid,sldlst_name,state_fips
0,Alaska,02001,State House District 1,02
1,Alaska,02002,State House District 2,02
2,Alaska,02003,State House District 3,02
3,Alaska,02004,State House District 4,02
4,Alaska,02005,State House District 5,02
5,Alaska,02006,State House District 6,02
6,Alaska,02007,State House District 7,02
7,Alaska,02008,State House District 8,02
8,Alaska,02009,State House District 9,02
9,Alaska,02010,State House District 10,02


In [13]:

# get Alaska election results data from the Alaska Division of Elections

# Fetch the JSON data
url = "https://www.elections.alaska.gov/enr/results/statewide.js"
response = requests.get(url, verify=False)
data = response.json()

# Process candidate names
candidate_names_df = pd.DataFrame(data['candidateNames'], columns=['Candidate'])
candidate_names_df.reset_index(inplace=True)
candidate_names_df.rename(columns={'index': 'candidateNumber'}, inplace=True)
candidate_names_df = candidate_names_df.astype({"Candidate": str})
candidate_names_df = candidate_names_df[
    candidate_names_df['Candidate'].isin(["Harris/Walz (DEM)", "Trump/Vance (REP)"])
]

# Process contest names
contest_names_df = pd.DataFrame(data['contestNames'], columns=['Contest'])
contest_names_df.reset_index(inplace=True)
contest_names_df.rename(columns={'index': 'contestNumber'}, inplace=True)
contest_names_df = contest_names_df.astype({"Contest": str})
contest_names_df = contest_names_df[
    contest_names_df['Contest'] == "U.S. President / Vice President"
]

# Process districts
districts = data['districts']

# Flatten the JSON to extract required information
rows = []
for district in districts:
    house_district = district['number']
    for contests in district['contests']:
        contestNumber = contests['n']
        candidates = contests['c']
        for candidate in candidates:
            candidateNumber = candidate['n']
            votes = candidate['t']
            row = {
                    'House District': house_district, 
                    'contestNumber': contestNumber, 
                    'candidateNumber': candidateNumber,
                    'votes': votes
            }
            rows.append(row)

# Create the dataframe
cadidate_contests_df = pd.DataFrame(rows)

cadidate_contests_df['House District'] = "State House District " + cadidate_contests_df['House District'].astype(str)

cadidate_contests_df = cadidate_contests_df.astype({
    "candidateNumber": int,
    "contestNumber": int,
    "votes": int
})

# Merge contests with districts
districts_contests_df = pd.merge(
    cadidate_contests_df, contest_names_df,
    on="contestNumber", how="right"
)

# Merge candidates with districts-contests
districts_contests_candidates_df = pd.merge(
    districts_contests_df, candidate_names_df,
    left_on="candidateNumber", right_on="candidateNumber", how="right"
)

# Remove unnecessary columns
districts_contests_candidates_df.drop(columns=["contestNumber", "candidateNumber", "Contest"], inplace=True)

# Group by House District and calculate total votes
total_votes_df = districts_contests_candidates_df.groupby("House District", as_index=False).agg({"votes": "sum"}).rename(columns={"votes": "total_votes"})

# Pivot table for party votes
pivot_df = districts_contests_candidates_df.pivot_table(
    index="House District",
    columns="Candidate",
    values="votes",
    aggfunc="sum"
).reset_index()
pivot_df.rename(columns={"Trump/Vance (REP)": "votes_gop", "Harris/Walz (DEM)": "votes_dem"}, inplace=True)

# Merge total votes with pivot table
merged_df = pd.merge(pivot_df, total_votes_df, on="House District", how="left")

# Merge with legislative districts
ak_election_results = pd.merge(
    merged_df, ak_sldlsts,
    left_on="House District", right_on="sldlst_name", how="left"
)

# Reorder columns and remove unnecessary ones
ak_election_results = ak_election_results[["state_name", "sldlst_name", "geoid", "total_votes", "votes_dem", "votes_gop"]]
ak_election_results.rename(columns={"geoid": "county_fips", "sldlst_name": "county_name"}, inplace=True)

# Calculate GOP percentage
ak_election_results["per_gop"] = ak_election_results["votes_gop"] / ak_election_results["total_votes"]

# Calculate DEM percentage
ak_election_results["per_dem"] = ak_election_results["votes_dem"] / ak_election_results["total_votes"]

# Calculate vote difference
ak_election_results["diff"] = ak_election_results["votes_gop"] - ak_election_results["votes_dem"]

# Calculate percentage point difference
ak_election_results["per_point_diff"] = ak_election_results["per_gop"] - ak_election_results["per_dem"]

# Ensure column types are correctly set
ak_election_results["county_fips"] = ak_election_results["county_fips"].astype(str)
ak_election_results["total_votes"] = ak_election_results["total_votes"].astype("Int64")
ak_election_results["votes_dem"] = ak_election_results["votes_dem"].astype("Int64")
ak_election_results["votes_gop"] = ak_election_results["votes_gop"].astype("Int64")

# Sort by county_fips
ak_election_results = ak_election_results.sort_values(by="county_fips").reset_index(drop=True)

print(ak_election_results.shape, '\n', ak_election_results.dtypes)
ak_election_results.head(100)

(40, 10) 
 state_name         object
county_name        object
county_fips        object
total_votes         Int64
votes_dem           Int64
votes_gop           Int64
per_gop           float64
per_dem           float64
diff                int64
per_point_diff    float64
dtype: object


,state_name,county_name,county_fips,total_votes,votes_dem,votes_gop,per_gop,per_dem,diff,per_point_diff
0,Alaska,State House District 1,02001,7536,3055,4481,0.594613,0.405387,1426,0.189225
1,Alaska,State House District 2,02002,7276,3626,3650,0.501649,0.498351,24,0.003299
2,Alaska,State House District 3,02003,10204,5906,4298,0.421207,0.578793,-1608,-0.157585
3,Alaska,State House District 4,02004,8739,6074,2665,0.304955,0.695045,-3409,-0.390090
4,Alaska,State House District 5,02005,5963,2530,3433,0.575717,0.424283,903,0.151434
5,Alaska,State House District 6,02006,10048,4226,5822,0.579419,0.420581,1596,0.158838
6,Alaska,State House District 7,02007,9283,2512,6771,0.729398,0.270602,4259,0.458796
7,Alaska,State House District 8,02008,10421,2554,7867,0.754918,0.245082,5313,0.509836
8,Alaska,State House District 9,02009,10795,5454,5341,0.494766,0.505234,-113,-0.010468
9,Alaska,State House District 10,02010,8004,3832,4172,0.521239,0.478761,340,0.042479


In [14]:
# combine Alaska election results with election results
all_election_results = pd.concat([election_results, ak_election_results], ignore_index=True)

# rename county_fips to geoid
all_election_results.rename(columns={"county_fips": "geoid"}, inplace=True)

# Sort by geoid
all_election_results = all_election_results.sort_values(by="geoid").reset_index(drop=True)

print(all_election_results.shape, '\n', all_election_results.dtypes)
all_election_results.head(100)

(3153, 10) 
 state_name         object
geoid              object
county_name        object
votes_gop           Int64
votes_dem           Int64
total_votes         Int64
diff                int64
per_gop           float64
per_dem           float64
per_point_diff    float64
dtype: object


,state_name,geoid,county_name,votes_gop,votes_dem,total_votes,diff,per_gop,per_dem,per_point_diff
0,Alabama,01001,Autauga County,20447,7429,28139,13018,0.726643,0.264011,0.462632
1,Alabama,01003,Baldwin County,95144,24763,120973,70381,0.786490,0.204699,0.581791
2,Alabama,01005,Barbour County,5578,4120,9766,1458,0.571165,0.421872,0.149293
3,Alabama,01007,Bibb County,7563,1617,9230,5946,0.819393,0.175190,0.644204
4,Alabama,01009,Blount County,25271,2569,28024,22702,0.901763,0.091671,0.810091
...,...,...,...,...,...,...,...,...,...,...
95,Alaska,02029,State House District 29,7315,2674,9989,4641,0.732306,0.267694,0.464611
96,Alaska,02030,State House District 30,7147,2665,9812,4482,0.728394,0.271606,0.456788
97,Alaska,02031,State House District 31,3473,3236,6709,237,0.517663,0.482337,0.035326
98,Alaska,02032,State House District 32,3739,1883,5622,1856,0.665066,0.334934,0.330132
